In [1]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta

In [24]:
#load sp500 data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

end_date = '2023-09-27'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

sp500 = sp500[pd.to_datetime(sp500["Date added"])<start_date]

symbols_list = sp500['Symbol'].unique().tolist()

df = yf.download(tickers=symbols_list, start=start_date, end=end_date).stack(future_stack=True)

df.columns = df.columns.str.lower()

[*********************100%%**********************]  345 of 345 completed


In [25]:
#calculate features

df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2 - (2*np.log(2)-1)*(np.log(df['adj close']) - np.log(df['open']))**2
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6


In [26]:
#aggregate to monthly level

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('Ticker')['dollar_volume'].resample('ME').mean().stack('Ticker', future_stack=True).to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('ME').last().stack('Ticker', future_stack=True)], axis=1).dropna()

In [27]:
#calculate 5year rolling average of dollar volume for each stock
data['dollar_volume'] = data['dollar_volume'].unstack('Ticker').rolling(5*12).mean().stack()
data['dollar_vol_rank'] = data.groupby('Date')['dollar_volume'].rank(ascending=False)
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)


In [28]:
#calculate mmonthly returns for different time horizons
def calculate_returns(df):
    outlier_cutoff = 0.005
    lags = [1,2,3,6,9,12]
    for lag in lags:
        df[f'return_{lag}m'] = (df['adj close']
                               .pct_change(lag)
                               .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                               .add(1)
                               .pow(1/lag)
                               .sub(1))
    return df
data = data.groupby('Ticker', group_keys=False).apply(calculate_returns).dropna()

In [31]:
#download fama franch factors and calculate rolling betas
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start='2010')[0].drop('RF', axis =1 )
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data = factor_data.join(data['return_1m']).sort_index()

C:\Users\Thoma\AppData\Local\Temp\ipykernel_28040\1216681836.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start='2010')[0].drop('RF', axis =1 )
C:\Users\Thoma\AppData\Local\Temp\ipykernel_28040\1216681836.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start='2010')[0].drop('RF', axis =1 )
C:\Users\Thoma\AppData\Local\Temp\ipykernel_28040\1216681836.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  factor_data = factor_data.resample('M').last().div(100)


In [38]:
factor_data.groupby(level=1).size()

Ticker
AAL     24
AAPL    24
ABBV    24
ABT     24
ACN     24
        ..
WFC     24
WMT     24
WYNN    24
XOM     24
ZTS     10
Length: 156, dtype: int64

In [35]:
factor_data

,Mkt-RF,SMB,HML,RMW,CMA,return_1m
Date,,,,,,
2021-10-31,0.0665,-0.0271,-0.0045,0.0164,-0.0144,0.058657
2021-11-30,-0.0155,-0.0176,-0.0044,0.0720,0.0171,0.105082
2021-12-31,0.0310,-0.0081,0.0325,0.0491,0.0439,0.074229
2022-01-31,-0.0625,-0.0406,0.1288,0.0087,0.0774,-0.015712
2022-02-28,-0.0229,0.0292,0.0311,-0.0210,0.0314,-0.054066
